In [2]:
import hsa
import numpy as np
import xarray as xr
import xarray.ufuncs as xu
from dask.diagnostics import ProgressBar
import scipy.stats as ss
import paths as ps
from datetime import datetime, timedelta
import bottleneck
import matplotlib.pyplot as plt
import seaborn as sns
import cfgrib

In [2]:
new_gefs = cfgrib.open_datasets(f'../temp/gefs_mean_024.grib2')


Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompatible with GRIB file
Ignoring index file '/E1/taylorm/espr/temp/gefs_mean_024.grib2.011ab.idx' incompat

In [56]:
[n for n in new_gefs if 't' in n][0]['t'].sel(isobaricInhPa=925)

<xarray.DataArray 't' (latitude: 361, longitude: 720)>
[259920 values with dtype=float32]
Coordinates:
    time           datetime64[ns] 2019-10-17T18:00:00
    step           timedelta64[ns] 1 days
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude      (longitude) float64 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
    valid_time     datetime64[ns] 2019-10-18T18:00:00
Attributes:
    GRIB_paramId:                             130
    GRIB_shortName:                           t
    GRIB_units:                               K
    GRIB_name:                                Temperature
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_dataType:                            pf
    GRIB_missingValue:                        9999
    GRIB_numberOfPoints:                      259920
    GRIB_totalNumber:                         20
    GRIB_typeOfLevel:            

In [7]:
variable = 'wnd'
with open(ps.log_directory + 'current_run.txt', "r") as f:
    model_date=datetime.strptime(f.readlines()[-1][5:13],'%Y%m%d')
mc_wnd = hsa.MClimate(model_date, variable, 24, percentage=10)
mc_mu = hsa.xarr_interpolate(mc_wnd.generate(type='mean',dask=True),gefs)

NameError: name 'gefs' is not defined

In [14]:
mc_mu=mc_wnd.generate(type='mean',dask=True)

In [28]:
mc_mu[[n for n in mc_mu.data_vars][0]]

<xarray.DataArray 'U-component_of_wind_height_above_ground' (time: 630, lat: 61, lon: 131)>
dask.array<sqrt, shape=(630, 61, 131), dtype=float32, chunksize=(1, 61, 131), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 20.0 21.0 22.0 23.0 24.0 ... 76.0 77.0 78.0 79.0 80.0
  * lon      (lon) float32 180.0 181.0 182.0 183.0 ... 307.0 308.0 309.0 310.0
  * time     (time) datetime64[ns] 2012-10-07 2012-10-07 ... 2012-10-27

In [22]:
xu.sqrt(mc_mu[[n for n in mc_mu.data_vars][0]]**2+mc_mu[[n for n in mc_mu.data_vars][1]]**2)

<xarray.DataArray (time: 630, lat: 61, lon: 131)>
dask.array<sqrt, shape=(630, 61, 131), dtype=float32, chunksize=(1, 61, 131), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 20.0 21.0 22.0 23.0 24.0 ... 76.0 77.0 78.0 79.0 80.0
  * lon      (lon) float32 180.0 181.0 182.0 183.0 ... 307.0 308.0 309.0 310.0
  * time     (time) datetime64[ns] 2012-10-07 2012-10-07 ... 2012-10-27

In [23]:
file = xr.open_dataset(f'../reforecast/tmp_mean_son.nc')


In [25]:
file.sel(pressure=925)

<xarray.Dataset>
Dimensions:       (fhour: 29, lat: 61, lon: 131, time: 3360)
Coordinates:
  * time          (time) datetime64[ns] 1989-08-21 1989-08-22 ... 2018-12-10
  * lat           (lat) float32 20.0 21.0 22.0 23.0 24.0 ... 77.0 78.0 79.0 80.0
  * lon           (lon) float32 180.0 181.0 182.0 183.0 ... 308.0 309.0 310.0
  * fhour         (fhour) timedelta64[ns] 0 days 00:00:00 ... 7 days 00:00:00
    pressure      float32 925.0
Data variables:
    intTime       (time) int32 ...
    intValidTime  (time, fhour) int32 ...
    Temperature   (time, fhour, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        Subset of data from 2nd-generation multi-decadal ensemble r...
    institution:  NOAA Earth System Research Laboratory (ESRL)
    source:       NCEP GFS v 9.01, T254L42.  Control initial conditions from ...
    references:   http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history:      Subset created 2019-10-03 05:02:08 UTC
    comment:     

In [3]:
import importlib
importlib.reload(hsa)

variable = 'tmp850'
hour = 24
lons = np.arange(180,310.1,0.5)
lats = np.arange(20,80.1,0.5)
with open(ps.log_directory + 'current_run.txt', "r") as f:
    model_date=datetime.strptime(f.readlines()[-1][5:13],'%Y%m%d')
mc_wnd = hsa.MClimate(model_date, variable, hour, percentage=10)
nfa = hsa.NewForecastArray('mean',variable,hour)
gefs = nfa.load_forecast(subset_lat=lats,subset_lon=lons)   
mc_mu = hsa.xarr_interpolate(mc_wnd.generate(type='mean',dask=True),gefs)

In [4]:
import importlib
importlib.reload(hsa)
percentiles = hsa.percentile(mc_mu, gefs)

> /E1/taylorm/espr/src/hsa.py(254)percentile()
-> try:
(Pdb) mclimate
<xarray.Dataset>
Dimensions:      (lat: 121, lon: 261, time: 630)
Coordinates:
  * time         (time) datetime64[ns] 2012-10-07 2012-10-07 ... 2012-10-27
  * lat          (lat) float64 80.0 79.5 79.0 78.5 78.0 ... 21.5 21.0 20.5 20.0
  * lon          (lon) float64 180.0 180.5 181.0 181.5 ... 309.0 309.5 310.0
Data variables:
    Temperature  (time, lat, lon) float64 262.7 262.7 262.7 ... 288.6 288.6
Attributes:
    Conventions:  CF-1.0
    title:        Subset of data from 2nd-generation multi-decadal ensemble r...
    institution:  NOAA Earth System Research Laboratory (ESRL)
    source:       NCEP GFS v 9.01, T254L42.  Control initial conditions from ...
    references:   http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history:      Subset created 2019-10-03 05:02:08 UTC
    comment:      Original dataset generated on DOE's supercomputers at Lawre...
(Pdb) forecast
<xarray.DataArray 't' (time: 1,

In [36]:
importlib.reload(hsa)
variable = 'wnd'
mc_wnd = hsa.MClimate(model_date, variable, hour)
mc_std = hsa.xarr_interpolate(mc_wnd.generate(type='sprd',dask=True),gefs)
subset = hsa.subset_sprd(percentiles, mc_std)
nfa_sprd = hsa.NewForecastArray('sprd', variable, hour)
gefs_sprd = nfa_sprd.load_forecast(subset_lat=lats,subset_lon=lons)
hsa_final = hsa.hsa_transform(gefs_sprd, subset)

In [37]:
hsa_final

<xarray.DataArray (lat: 121, lon: 261)>
dask.array<arctanh, shape=(121, 261), dtype=float32, chunksize=(121, 261), chunktype=numpy.ndarray>
Coordinates:
    time        datetime64[ns] 2019-10-17T18:00:00
    step        timedelta64[ns] 1 days
  * lat         (lat) float64 80.0 79.5 79.0 78.5 78.0 ... 21.5 21.0 20.5 20.0
  * lon         (lon) float64 180.0 180.5 181.0 181.5 ... 309.0 309.5 310.0
    valid_time  datetime64[ns] 2019-10-18T18:00:00

In [38]:
gefs_sprd

<xarray.DataArray (lat: 121, lon: 261)>
array([[0.4031129 , 0.38183767, 0.38209945, ..., 0.31622776, 0.3       ,
        0.29832867],
       [0.45276928, 0.43863425, 0.4313931 , ..., 0.29068884, 0.2842534 ,
        0.286007  ],
       [0.5166237 , 0.5166237 , 0.5240229 , ..., 0.3522783 , 0.3466987 ,
        0.33286634],
       ...,
       [0.4103657 , 0.49517673, 0.63890535, ..., 0.64140475, 0.64637446,
        0.6720119 ],
       [0.44821867, 0.5024938 , 0.60539246, ..., 0.5547071 , 0.5824946 ,
        0.62681735],
       [0.48010415, 0.47413078, 0.53712195, ..., 0.48259717, 0.49648768,
        0.52153623]], dtype=float32)
Coordinates:
    time        datetime64[ns] 2019-10-17T18:00:00
    step        timedelta64[ns] 1 days
  * lat         (lat) float64 80.0 79.5 79.0 78.5 78.0 ... 21.5 21.0 20.5 20.0
  * lon         (lon) float64 180.0 180.5 181.0 181.5 ... 309.0 309.5 310.0
    valid_time  datetime64[ns] 2019-10-18T18:00:00

In [39]:
nfa_mean = hsa.NewForecastArray('mean', variable, hour)
gefs_mean = nfa_mean.load_forecast(subset_lat=lats,subset_lon=lons)

In [72]:
import plot as plot
importlib.reload(plot)

<module 'plot' from '/E1/taylorm/espr/src/plot.py'>